In [433]:
%matplotlib inline
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import Tensor,optim
import numpy as np
import scipy.io
import pickle
import h5py
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import tqdm
import itertools
from scipy.signal import butter,lfilter, filtfilt, freqz, find_peaks_cwt, medfilt, find_peaks, resample
from trial import Trial
from tqdm import tnrange
import os
import sys
import pandas as pd
import pingouin as pg
np.random.seed(sum(map(ord, "aesthetics")))

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

from pymer4.simulate import simulate_lmm, simulate_lm
from pymer4.models import Lm, Lmer, Lm2
from sklearn.preprocessing import StandardScaler

#os.getcwd()

# print(f"Is CUDA supported by this system?{torch.cuda.is_available()}")
# print(f"CUDA version: {torch.version.cuda}")
  
# # Storing ID of current CUDA device
# cuda_id = torch.cuda.current_device()
# print(f"ID of current CUDA device:{torch.cuda.current_device()}")    
# print(f"Name of current CUDA device:{torch.cuda.get_device_name(cuda_id)}")
    
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device)

In [434]:
def get_by_animal(trial_list, name):
    return list(filter(lambda trial: trial.name==name,trial_list))

def get_sessions(trial_list):
    sessions = list()
    for trial in trial_list:
        if not trial.session in sessions:
            sessions.append(trial.session)
    sessions.sort()
    return sessions

def get_by_session(trial_list, session):
    return list(filter(lambda trial: trial.session==session,trial_list))

def get_by_date(trial_list, date):
    date = str(date)
    return list(filter(lambda trial: trial.date==date,trial_list))

def get_by_session_type(trial_list, session_type):
    return list(filter(lambda trial: trial.session_type==session_type,trial_list))

def get_performance(trial_list):
    total = 0.0
    for trial in trial_list:
        total += int(trial.correct)
    return total/len(trial_list)

def get_correct_trials(trial_list):
    total = 0.0
    for trial in trial_list:
        if trial.correct == True:
            print(1)
            
        else:
            print(0)

def get_by_delay(trial_list, delay):
    return list(filter(lambda trial: trial.delay_time==delay,trial_list))

def get_by_contrast(trial_list, contrast):
    return list(filter(lambda trial: round(trial.high_count/trial.low_count)==contrast,trial_list))

def get_performance_list(trials):
    sessions = get_sessions(trials)
    performance_list = list()
    for session in sessions:
        performance_list.append(get_performance(get_by_session(trials,session)))
    return performance_list

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y

def get_inhalation(trial):
    breath = butter_lowpass_filter(trial.breath_profile, cutoff = 10.0, fs = 1000.0, order = 6)
    breath = breath - breath.mean()
    breath = breath / (breath.max() - breath.min()) * 2
    inhalation = np.gradient(breath) < -0.00
    return inhalation.astype(float)

def get_odor_profile_actual(trial):
    kernel = np.load('odor_kernel_50ms.npy')
    pulse_onsets = (np.diff(trial.odor_profile)>0).astype(float)
    pulse_onsets_shifted = np.concatenate((np.zeros((70,)),pulse_onsets,np.zeros(131)))
    odor_profile_actual = np.convolve(pulse_onsets_shifted,kernel,mode='same')
    return odor_profile_actual

def get_full_inhalation(trial):
    breath = butter_lowpass_filter(np.concatenate((trial.pre_breath_profile,trial.breath_profile)), 
                                   cutoff = 10.0, fs = 1000.0, order = 6)
    breath = breath - breath.mean()
    breath = breath / (breath.max() - breath.min()) * 2
    inhalation = np.gradient(breath) < -0.00
    return inhalation.astype(float)

def get_full_inhalation_test(trial):
    breath = butter_lowpass_filter(np.concatenate((trial.pre_breath_profile,trial.breath_profile)), 
                                   cutoff = 10.0, fs = 1000.0, order = 6)
    breath = breath - breath.mean()
    breath = breath / (breath.max() - breath.min()) * 2
    breath[breath>0] = 0
    return -breath
    

def get_inhaled_odor(trial):
    inhalation = get_inhalation(trial)
    odor_profile = get_odor_profile_actual(trial)
    total_odor = (inhalation[0:odor_profile.size]*odor_profile).sum()
    return total_odor

def get_inhaled_odor_trace(trial):
    sniff_mod_kernel = np.load('inhalation_kernel_fine_weights_active.npy')

    delay_time = trials[0].delay_time
    sniff = butter_lowpass_filter(trials[0].breath_profile,10,1000,3)[0:delay_time]
    sniff = (sniff - sniff.mean() +1)/2
    sniff_onset,_ = find_peaks(sniff,distance=150)
    num_sniffs = len(sniff_onset)
    odor_profile_actual = get_odor_profile_actual(trial)
    sniff_norm = np.zeros((num_sniffs,250))
    odor_norm = np.zeros((num_sniffs,250))

    sniff_marks = np.concatenate((sniff_onset[:-1].reshape(-1,1),sniff_onset[1:].reshape(-1,1)),axis=1)
    sniff_durations = np.diff(sniff_marks,axis=1)
    num_sniffs = sniff_marks.shape[0]
    sniff_mod = np.zeros(sniff.shape)
    for i in range(num_sniffs):
        sniff_mod[int(sniff_marks[i,0]):int(sniff_marks[i,1])] = resample(sniff_mod_kernel,int(sniff_durations[i]))
        
    inhaled_odor_trace =  odor_profile_actual[0:delay_time]*sniff_mod
    return inhaled_odor_trace

def get_by_pulses(trial_list, low_pulses, high_pulses):
    return list(filter(lambda trial: (trial.num_pulses>=low_pulses and trial.num_pulses<high_pulses),
                       trial_list))

def get_by_inhaled(trial_list, low, high):
    return list(filter(lambda trial: (get_inhaled_odor(trial)>=low and get_inhaled_odor(trial)<high),
                       trial_list))

In [435]:
all_trials_BNPS = pickle.load(open('./Trials/all_trials_BNPS.pickle','rb'))
all_trials_RCS = pickle.load(open("./Trials/all_trials_RCS.pickle",'rb'))
merged_dataset = all_trials_BNPS + all_trials_RCS
#stark_exc = pickle.load(open('./Trials/Stark_20220223_3.pickle','rb'))
#n_exc = pickle.load(open('./Trials/Natasha_004.pickle','rb'))

In [436]:

random_trials_all = get_by_session_type(all_trials_RCS, "random")
#block_trials_all = get_by_session_type(merged_dataset, "block")

In [ ]:
animal_list = ['Ragdoll', 'Siamese', 'Calico']

sessions_Ragdoll = get_sessions(get_by_animal(all_trials_RCS, 'Ragdoll'))
sessions_Calico = get_sessions(get_by_animal(all_trials_RCS, 'Calico'))
sessions_Siamese = get_sessions(get_by_animal(all_trials_RCS, 'Siamese'))

for i in range(len(sessions_Ragdoll)):
    t = get_by_session(get_by_animal(all_trials_RCS, 'Ragdoll'), i)
    print(t[0].name, t[0].session, np.round(t[0].high_count/t[0].low_count), t[0].high_count, t[0].low_count, t[0].delay_time)

Ragdoll 0 10.0 15.0 1.5 1500
Ragdoll 1 10.0 15.0 1.5 1500
Ragdoll 2 8.0 12.5 1.6666666666666667 2500
Ragdoll 3 6.0 14.0 2.3333333333333335 3500
Ragdoll 4 6.0 20.0 3.3333333333333335 5000
Ragdoll 5 3.0 20.0 5.833333333333333 7000
Ragdoll 6 3.0 25.0 8.333333333333334 10000
Ragdoll 7 3.0 22.22222222222222 8.333333333333334 10000
Ragdoll 8 3.0 25.0 8.333333333333334 10000
Ragdoll 9 3.0 25.0 8.333333333333334 10000
Ragdoll 10 3.0 25.0 8.333333333333334 10000
Ragdoll 11 3.0 25.0 8.333333333333334 10000
Ragdoll 12 3.0 12.5 4.166666666666667 5000
Ragdoll 13 3.0 12.5 4.166666666666667 5000
Ragdoll 14 3.0 6.25 2.0833333333333335 2500
Ragdoll 15 3.0 3.125 1.0416666666666667 1250
Ragdoll 16 4.0 4.166666666666667 1.0416666666666667 1250
Ragdoll 17 4.0 8.333333333333334 2.0833333333333335 2500
Ragdoll 18 2.0 8.333333333333334 4.166666666666667 5000


In [ ]:
ttrials = get_by_session_type(all_trials_RCS, 'random')



for i in range(len(ttrials)):
    trial = ttrials[i]
    print(trial.name)

Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll
Ragdoll


In [157]:
class Model(nn.Module):
    
    def __init__(self):
        
        super(Model, self).__init__()
        
    def build_model(self, input_dim, output_dim):
        model = torch.nn.Sequential()#.#to(device)
        # computes w_c^T x + b_c 
        model.add_module("linear1", torch.nn.Linear(input_dim, output_dim,bias=True))
        # Compute our log softmax term.
        model.add_module("softmax", torch.nn.LogSoftmax(dim=1)) #We used softmax because is a 
        #generalized version of the sigmoid
        return model

    def train(self, model, loss, optimizer, x_val, y_val):

        # Take in x and y and make variable.
        x = Variable(x_val)#.to(device)
        y = Variable(y_val)#.to(device)
        
        #model#.to(device)
        # Resets the gradients to 0
        optimizer.zero_grad()

        # Computes the function above. (log softmax w_c^T x + b_c)
        fx = model.forward(x)

        # Computes a loss. Gives a scalar. 
        output = loss.forward(fx, y)

        # Magically computes the gradients. 
        output.backward()

        # updates the weights
        optimizer.step()
        return output.item()

    def predict(self, model, x_val):
        #model#.to(device)
        x = Variable(x_val, requires_grad=False)
        output = model.forward(x)
        return output.data.argmax(axis=1)   

In [158]:
def DFcompiler(arr1, delay):

    df_list = ['Validated', 'Shuffled', 'Truth']
    df_dict = {}
    for i in df_list:
        df_dict[i] = pd.DataFrame()

    for i, j in zip((range(len(arr1))), df_list):

        df = pd.DataFrame(data=[(*axes, v.item()) for axes, v in zip(itertools.product(*[range(i) for i in arr1.shape]), np.nditer(arr1))], columns=tuple('xyvwu')) #z
    
    for k, types in zip((range(len(df['x'].unique()))), df_list):
        df=df.replace({'x':{k:types}})
    
    #con = range(len(df['y'].unique())).astype(int)
    
    for c, contr in zip((range(len(df['y'].unique()))), ctst):
        df=df.replace({'y':{c:contr}})
        
#     for a, anim in zip((range(len(df['z'].unique()))), animales):
#         df=df.replace({'z':{a:anim}})

    df=df.rename(columns={'x':'Dtype','t':'Delay','y':'Contrast','v':'Repeat','w':'Bin', 'u': "Weight"}) #'z':'Animal',
    df.insert(1,'Delay',delay)

    return df

In [490]:
class AutoLogReg(Model):
    
    def __init__(self, data, trainsize, odor_k=True, inh_k=False, full_inh=False):
        
        super(Model).__init__()
        #self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.data = data
        self.trainsize = trainsize
        self._use_odor_kernel = odor_k
        self._use_inh_kernel = inh_k
        self._full_inhalation = full_inh
        self.nseg = 5
        self.x_raw = None
        self.sniff = None
        
    def main(self, target,x_test,z_test,x_train,z_train):

        torch.manual_seed(42)  
        mod=Model()
        #mod.to(self.device)
        #print(self.nsamples, self.nseg)
        n_examples, n_features = self.nsamples, self.nseg
        n_classes = 2
        model = mod.build_model(n_features, n_classes)

        # Loss here is negative log-likelihood 
        loss = torch.nn.NLLLoss()

        # Optimizer. SGD stochastic gradient. 
        optimizer = optim.Adam(model.parameters(), lr=0.005)

        batch_size = self.nsamples
        acc = 0
        i = 0
        
        while(i<1000):
            i += 1
            cost = 0.
            cost += mod.train(model, loss, optimizer,
                              x_train, z_train)
            predz = mod.predict(model, x_train)
            predz_test = mod.predict(model,x_test)
            acc = (100.0*(predz == z_train)).mean()
            acc_test = (100.0*(predz_test == z_test)).mean()
            if i%10 == 0:
                #print('epoch =',i,',cost =  %.2f' % cost, 'train performance = %.2f%%'%acc,
                #'test performance = %.2f%%' %acc_test)
                pass
        return model, acc, acc_test, cost

    def validate(self, n,target): # x_array (odor profile passed through the kernel) 
        #prediction on z_array (licking decision)

        models = list()
        avg_acc = 0
        avg_acc_test = 0
        avg_cost = 0
        for i in tqdm.notebook.trange(n):
            ind_array = np.random.permutation(self.nsamples)
            x_shuffled = self.x_array[ind_array,:]
            z_shuffled = self.z_array[ind_array].squeeze()
            x_all = Tensor(x_shuffled.astype(float))#.to(self.device)
            z_all = Tensor(z_shuffled.astype(float))#.to(self.device)
            z_all = z_all.long()#.to(self.device)
            x_train = x_all[0:int(self.nsamples*self.trainsize),:]
            z_train = z_all[0:int(self.nsamples*self.trainsize)]
            x_test = x_all[int(self.nsamples*self.trainsize):,:]
            z_test = z_all[int(self.nsamples*self.trainsize):]
            model, acc, acc_test,cost = self.main(target,x_test,z_test,x_train,z_train)
            #model.to(device)
            avg_acc += acc
            avg_acc_test += acc_test
            avg_cost += cost
            models.append(model)

        avg_acc/=n
        avg_acc_test/=n
        avg_cost/=n

        print("")
        print('validation rounds =',i+1,'train performance = %.2f%%'%avg_acc,
              'test performance = %.2f%%' %avg_acc_test,'cost = %.2f' %avg_cost)
        return models

    def validate_truth(self,n,target): # x_array (odor profile passed through the kernel) 
        #prediction on y_array (reward side)
        models = list()
        avg_acc = 0
        avg_acc_test = 0
        for i in tqdm.notebook.trange(n):
            ind_array = np.random.permutation(self.nsamples)
            x_shuffled = self.x_array[ind_array,:]
            z_shuffled = self.y_array[ind_array].squeeze()
            x_all = Tensor(x_shuffled.astype(float))#.to(self.device)
            z_all = Tensor(z_shuffled.astype(float))#.to(self.device)
            z_all = z_all.long()
            x_train = x_all[0:int(self.nsamples*self.trainsize),:]
            z_train = z_all[0:int(self.nsamples*self.trainsize)]
            x_test = x_all[int(self.nsamples*self.trainsize):,:]
            z_test = z_all[int(self.nsamples*self.trainsize):]
            model, acc, acc_test,cost = self.main(target,x_test,z_test,x_train,z_train)
            #model.to(device)
            avg_acc += acc
            avg_acc_test += acc_test
            models.append(model)
        avg_acc/=n
        avg_acc_test/=n
 
        print('validation rounds =',i+1,'train performance = %.2f%%'%avg_acc,'test performance = %.2f%%' %avg_acc_test)
        return models

    def validate_shuffled(self,n,target): #x_array (odor profile passed through the kernel) in which the time
        # course of the odor profile has been shuffled for each trial. The prediction is again on
        #z_array (licking decision, unshuffled)

        models = list()
        avg_acc = 0
        avg_acc_test = 0

        for i in tqdm.notebook.trange(n):
            x_raw_shuffled = np.zeros(self.x_raw.shape)
            for j in range(self.nsamples):
                ind_array = np.random.permutation(self.delay)
                x_raw_shuffled[j,:] = self.x_raw[j,ind_array]

            x_shuffled2 = x_raw_shuffled.reshape((self.nsamples,self.nseg,int(self.delay/self.nseg))).mean(axis=2).squeeze()
            ind_array = np.random.permutation(self.nsamples)
            x_shuffled = x_shuffled2[ind_array,:]
            z_shuffled = self.z_array[ind_array].squeeze()
            x_all = Tensor(x_shuffled.astype(float))#.to(self.device)
            z_all = Tensor(z_shuffled.astype(float))#.to(self.device)
            z_all = z_all.long()
            x_train = x_all[0:int(self.nsamples*self.trainsize),:]
            z_train = z_all[0:int(self.nsamples*self.trainsize)]
            x_test = x_all[int(self.nsamples*self.trainsize):,:]
            z_test = z_all[int(self.nsamples*self.trainsize):]
            model, acc, acc_test,cost = self.main(target,x_test,z_test,x_train,z_train)
            #model.to(device)
            avg_acc += acc
            avg_acc_test += acc_test
            models.append(model)
        avg_acc/=n
        avg_acc_test/=n

        print("")
        print('validation rounds =',i+1,'train performance = %.2f%%'%avg_acc,'test performance = %.2f%%' %avg_acc_test)
        return models
    
    def get_inhaled_odor_trace(self, trial):
        sniff_mod_kernel = np.load('inhalation_kernel_fine_weights_active.npy')

        #delay_time = trials[0].self.delay
        #sniff = butter_lowpass_filter(trials[0].breath_profile,10,1000,3)[0:self.delay]
        self.sniff = (self.sniff - self.sniff.mean() +1)/2
        sniff_onset,_ = find_peaks(self.sniff,distance=150)
        num_sniffs = len(sniff_onset)
        odor_profile_actual = get_odor_profile_actual(trial)
        sniff_norm = np.zeros((num_sniffs,250))
        odor_norm = np.zeros((num_sniffs,250))

        sniff_marks = np.concatenate((sniff_onset[:-1].reshape(-1,1),sniff_onset[1:].reshape(-1,1)),axis=1)
        sniff_durations = np.diff(sniff_marks,axis=1)
        num_sniffs = sniff_marks.shape[0]
        sniff_mod = np.zeros(self.sniff.shape)
        for i in range(num_sniffs):
            sniff_mod[int(sniff_marks[i,0]):int(sniff_marks[i,1])] = resample(sniff_mod_kernel,int(sniff_durations[i]))
            inhaled_odor_trace =  odor_profile_actual[0:trial.delay_time]*sniff_mod
            return inhaled_odor_trace
    
    
    def autologreg(self, nrounds): #def autologreg(self, nrounds, binsize):
        
        #self.nseg = self.delay//binsize
        #self.nseg = 5

        #print(nseg)
        all_validated_weights = np.nan*np.ones((len(self.contrasts),nrounds,self.nseg)) #len(self.animals),
        all_shuffled_weights = np.nan*np.ones((len(self.contrasts),nrounds,self.nseg)) #len(self.animals),
        all_reward_weights = np.nan*np.ones((len(self.contrasts),nrounds,self.nseg)) #len(self.animals),

        td = get_by_delay(self.data,self.delay)
        
#         print(self.delay)
#         if self.delay == 1250 or 5000:
#             self.delay = self.delay+1
#             print('the delay is', self.delay)

#         elif self.delay == 2500 or 10000:
#             self.delay = self.delay+2
#             print('the delay is', self.delay)


        for c in range(len(self.contrasts)):

            print('the contrast is:', self.contrasts[c])

            trials = get_by_contrast(td,self.contrasts[c])

            #for animal, animal_index in zip(self.animals, range(all_validated_weights.shape[1])): 
                #Selection of the data for running the logistic regression analysis

             #   print(animal)
              #  trials = get_by_animal(trialx, animal)
    
            self.nsamples = len(trials)

            if self.nsamples != 0:
                print(self.nsamples)
                dt=0
                pre_time = trials[0].refract_time
                print(all_validated_weights.shape)

                self.x_array = np.zeros((self.nsamples,self.nseg)) # Each row is a trial, each cell is a 1 second 
                #timepoint
                self.x_raw = np.zeros((self.nsamples,self.delay))
                self.y_array = np.zeros((self.nsamples,)) # Each cell is a trial
                self.z_array = np.zeros((self.nsamples,)) # Each cell is a trial

                for i in range(self.nsamples): #looping over the lenght of the trial list

                    #splitters = int(self.delay/self.nseg)*np.arange(1,self.nseg)
                    #print(self.delay)

                    self.y_array[i] = trials[i].reward_side #Stores the value of the correct side for each trial
                    self.z_array[i] = trials[i].lick_side #Stores the value of the mice decision for each trial
                    self.sniff = butter_lowpass_filter(trials[0].breath_profile,10,1000,3)[0:self.delay]

                    if self._use_odor_kernel:
                        self.x_raw[i,:] = get_odor_profile_actual(trials[i])[0:self.delay] #allocates odor pulse 
                                                #profile for each trial into x_raw
                            #print('we will be making predictions based on data that went through odor kernel')

                    elif self._use_inh_kernel:

                        self.x_raw[i,:] = self.get_inhaled_odor_trace(trials[i])
                            #print('we will be making predictions based on data that went through inhalation kernel')

                    elif self._full_inhalation:
                        self.x_raw[i,:] = get_full_inhalation(trials[i])[(
                            pre_time+dt):(pre_time+dt+self.delay)] * get_odor_profile_actual(trials[i])[0:self.delay]
                        #print('we will be making predictions based on data that went through get_full_inhalation 
                        #method')

                #dup_array = np.array_split(get_odor_profile_actual(trials[i])[0:self.delay], splitters)

                self.x_array = self.x_raw.reshape((self.nsamples,self.nseg, int(self.delay/self.nseg))).mean(axis=2).squeeze()  #(int(self.delay/self.nseg))
                #we take the mean for parameter reduction

                ###############################################################################################
                # Now I will run the logistic regression analysis to see how good pulse number predicts licking
                ###############################################################################################

                models = self.validate(nrounds,80) #Where this 80 comes from (target)
                validated_weights = torch.zeros((nrounds, self.nseg))

                for i,model in enumerate(models):
                    validated_weights[i,:] = model.linear1.weight[1].detach()

                all_validated_weights[c,:,:] = validated_weights #animal_index

                mean_validated_weights = validated_weights.mean(axis=0)
                std_validated_weights = validated_weights.std(axis=0)

                ###############################################################################################
                # Same as above but with the temporal profile of odor pulses shuffled for each trial
                ###############################################################################################

                shuffled_models = self.validate_shuffled(nrounds,80)
                weights_shuffled = torch.zeros((nrounds, self.nseg))

                for i,model in enumerate(shuffled_models):
                    weights_shuffled[i,:] = model.linear1.weight[1].detach()

                mean_weights_shuffled = weights_shuffled.mean(axis=0)
                std_weights_shuffled = weights_shuffled.mean(axis=0)

                all_shuffled_weights[c,:,:] = weights_shuffled #animal_index

                ###############################################################################################
                # Logistic regression using pulses to predict reward locations (~ideal observer)
                ###############################################################################################    

                reward_models = self.validate_truth(nrounds, 80)
                weights_reward = torch.zeros((nrounds, self.nseg))

                for i,model in enumerate(reward_models):
                    weights_reward[i,:] = model.linear1.weight[1].detach()

                mean_weights_reward = weights_reward.mean(axis=0) #shuffled
                std_weights_reward = weights_reward.mean(axis=0) #shuffled

                all_reward_weights[c,:,:] = weights_reward #animal_index

            else:
                pass

        results = np.stack((all_validated_weights, all_shuffled_weights, all_reward_weights))
        return results
    
    def compiler_df(self): #def autologreg(self, nrounds, binsize):

        results_list=list()
        total_trials = get_by_session_type(self.data, 'random')
        
        for i in range(len(total_trials)):

            trial = total_trials[i]

            dt=0
            pre_time = trial.refract_time
            self.sniff = butter_lowpass_filter(trial.breath_profile,10,1000,3)[0:trial.delay_time]

            if self._use_odor_kernel:
                self.x_raw = get_odor_profile_actual(trial)[0:trial.delay_time] #allocates odor pulse 
                                        #profile for each trial into x_raw
                    
            elif self._use_inh_kernel:

                self.x_raw = self.get_inhaled_odor_trace(trial)
                    
            elif self._full_inhalation:
                self.x_raw = get_full_inhalation(trial)[(
                    pre_time+dt):(pre_time+dt+trial.delay_time)] * get_odor_profile_actual(trial)[0:trial.delay_time]
            
            test = self.x_raw.reshape(self.nseg,int(trial.delay_time/self.nseg))
            #print(test[0, 0:3])
            #print('###')
            #print(test.mean(axis=1))
                
            self.new_x_raw = self.x_raw.reshape(self.nseg,int(trial.delay_time/self.nseg)).mean(axis=1).squeeze()
            #print(self.new_x_raw)
              #(int(self.delay/self.nseg))
            #we take the mean for parameter reduction
            keys = []
            for i in range(len(self.new_x_raw)):
                keys.append('Pulses_bin_'+str(i+1))
            
            pulse_dicts = {}
            for i, j in zip(keys, range(len(self.new_x_raw))):
                print(i)
                print(self.new_x_raw[j])
                pulse_dicts[i] = self.new_x_raw[j]
                

            result_row_1 = {'Animal': trial.name,
                'Window':trial.delay_time,
                'Difficulty':np.round(trial.high_count/trial.low_count),
                'Session': trial.session,
                'Trial': trial.number,
                'Reward_side': trial.reward_side,
                'Choice': trial.lick_side}
            
            result_row_1.update(pulse_dicts)


            results_list.append(result_row_1)
    
        result_df = pd.DataFrame(results_list)
        return result_df

In [491]:
animales = ['Siamese']#, 'Natasha', 'Peterparker', 'Stark']
contrastes = [3] #[2,3,4,6]
delays = [1250, 2500, 5000, 10000]
ctst = ['1:3'] #'1:2','1:3','1:4','1:6'
dts = ['d5000'] #'d1250', 'd2500' 'd2500','d5000', 'd10000'
df_dicts = {}

for i in dts:
    df_dicts[i] = pd.DataFrame()
    
inh_kernel_4bins = AutoLogReg(random_trials_all, 0.8, odor_k=True, inh_k=False, full_inh=False)

# for i, q in zip((range(len(inh_kernel_4bins))), dts):
#     arrax = inh_kernel_4bins[i].autologreg2(200)
    #df_dicts[q] = DFcompiler(arrax, inh_kernel_4bins[i].delay)
    
#df_3b_inhk = pd.concat(df_dicts, ignore_index = True)

arrax = inh_kernel_4bins.compiler_df()

Pulses_bin_1
0.0
Pulses_bin_2
0.0
Pulses_bin_3
0.12698851479209664
Pulses_bin_4
0.002575069032159363
Pulses_bin_5
0.12671978781887722
Pulses_bin_1
0.0
Pulses_bin_2
0.0
Pulses_bin_3
0.129563583824256
Pulses_bin_4
0.0
Pulses_bin_5
0.0
Pulses_bin_1
0.6423478211827648
Pulses_bin_2
0.1350336817627712
Pulses_bin_3
0.35117585531873063
Pulses_bin_4
0.5468875975254971
Pulses_bin_5
0.26800880157407636
Pulses_bin_1
0.09833878538066183
Pulses_bin_2
0.031224798443594153
Pulses_bin_3
0.0
Pulses_bin_4
0.129563583824256
Pulses_bin_5
0.0
Pulses_bin_1
0.0
Pulses_bin_2
0.0
Pulses_bin_3
0.12642676506471726
Pulses_bin_4
0.0031368187595387604
Pulses_bin_5
0.11026688910273245
Pulses_bin_1
0.12896417302070626
Pulses_bin_2
0.0600674268975637
Pulses_bin_3
0.070095567730242
Pulses_bin_4
0.0
Pulses_bin_5
0.0
Pulses_bin_1
0.129563583824256
Pulses_bin_2
0.0
Pulses_bin_3
0.12956358382425598
Pulses_bin_4
0.0
Pulses_bin_5
0.0
Pulses_bin_1
0.12956358382425598
Pulses_bin_2
0.25912716764851196
Pulses_bin_3
0.768833346051

In [492]:
df2 = arrax.loc[(arrax['Difficulty'] <7) & (arrax['Window'].isin(delays))]
df2


,Animal,Window,Difficulty,Session,Trial,Reward_side,Choice,Pulses_bin_1,Pulses_bin_2,Pulses_bin_3,Pulses_bin_4,Pulses_bin_5
448,Siamese,5000,3.0,12,0,0,0.0,0.077738,0.000000,0.000000,0.038869,0.037040
449,Siamese,5000,3.0,12,1,0,0.0,0.038869,0.038869,0.077738,0.000000,0.000000
450,Siamese,5000,3.0,12,2,1,1.0,0.077003,0.117045,0.116904,0.077738,0.116607
451,Siamese,5000,3.0,12,3,0,0.0,0.038869,0.000000,0.077738,0.000000,0.036590
452,Siamese,5000,3.0,12,4,1,0.0,0.077738,0.115978,0.039425,0.116680,0.154210
...,...,...,...,...,...,...,...,...,...,...,...,...
8409,Calico,10000,3.0,8,131,0,0.0,0.038869,0.019435,0.058304,0.023701,0.015168
8410,Calico,10000,3.0,8,132,1,1.0,0.038869,0.058724,0.077318,0.077738,0.194345
8411,Calico,10000,3.0,8,133,1,0.0,0.116607,0.095965,0.117815,0.019435,0.077864
8412,Calico,10000,3.0,8,134,0,0.0,0.019153,0.019716,0.058121,0.039052,0.038869


In [493]:

scaler = StandardScaler()
continuous_cols = ['Pulses_bin_1', 'Pulses_bin_2', 'Pulses_bin_3', 'Pulses_bin_4', 'Pulses_bin_5']
print(continuous_cols)
df2[continuous_cols] = scaler.fit_transform(df2[continuous_cols])
df2


['Pulses_bin_1', 'Pulses_bin_2', 'Pulses_bin_3', 'Pulses_bin_4', 'Pulses_bin_5']


/var/folders/xy/wcn_mfbd55s9g3lg4nhmyp_m0000gn/T/ipykernel_73086/734896103.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[continuous_cols] = scaler.fit_transform(df2[continuous_cols])


,Animal,Window,Difficulty,Session,Trial,Reward_side,Choice,Pulses_bin_1,Pulses_bin_2,Pulses_bin_3,Pulses_bin_4,Pulses_bin_5
448,Siamese,5000,3.0,12,0,0,0.0,0.188742,-1.001322,-1.035801,-0.455953,-0.478512
449,Siamese,5000,3.0,12,1,0,0.0,-0.409153,-0.445461,0.138981,-1.030881,-1.018601
450,Siamese,5000,3.0,12,2,1,1.0,0.177441,0.672516,0.730864,0.118975,0.681663
451,Siamese,5000,3.0,12,3,0,0.0,-0.409153,-1.001322,0.138981,-1.030881,-0.485079
452,Siamese,5000,3.0,12,4,1,0.0,0.188742,0.657268,-0.440007,0.694979,1.229950
...,...,...,...,...,...,...,...,...,...,...,...,...
8409,Calico,10000,3.0,8,131,0,0.0,-0.409153,-0.723391,-0.154714,-0.680314,-0.797430
8410,Calico,10000,3.0,8,132,1,1.0,-0.409153,-0.161522,0.132632,0.118975,1.815172
8411,Calico,10000,3.0,8,133,1,0.0,0.786637,0.371063,0.744618,-0.743417,0.116745
8412,Calico,10000,3.0,8,134,0,0.0,-0.712434,-0.719362,-0.157477,-0.453249,-0.451846


In [541]:
df3 = df2.loc[(df2['Window'] == 5000)]# & (df2['Contrast'] == 3.0)]
df3['Difficulty'].unique()

array([3., 6., 2.])

In [535]:
m0 = Lmer(formula = 'Choice ~ 1', family='binomial', data = df3)
m1 = Lmer(formula = 'Choice ~ 1 + (1| Animal) + (1| Session)', family='binomial', data = df3)
m2 = Lmer(formula = 'Choice ~ Pulses_bin_1 + Pulses_bin_2 + Pulses_bin_3 + Pulses_bin_4 + Pulses_bin_5 + (1|Animal) + (1|Session)', family='binomial', data = df3)
m3 = Lmer(formula = 'Choice ~ Pulses_bin_1:Difficulty + Pulses_bin_2:Difficulty + Pulses_bin_3:Difficulty + Pulses_bin_4:Difficulty + Pulses_bin_5:Difficulty + (1| Animal) + (1| Session)', family='binomial', data = df3)
m4 = Lmer(formula = 'Choice ~ Pulses_bin_1*Difficulty + Pulses_bin_2*Difficulty + Pulses_bin_3*Difficulty + Pulses_bin_4*Difficulty + Pulses_bin_5*Difficulty + (1|Animal) + (1|Session)', family='binomial', data = df3)
#m5 = Lmer(formula = 'Choice ~ Pulses*Bin + Difficulty + (1| Animal)', family='binomial', data = df3)





In [547]:
print(m4.fit(factors={"Difficulty": ['2.0','3.0','6.0']}, summarize=True, control="optimizer='bobyqa'")) #factors={"Difficulty": ['2.0','3.0','4.0','6.0']}, 


boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

boundary (singular) fit: see help('isSingular') 

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: Choice~Pulses_bin_1*Difficulty+Pulses_bin_2*Difficulty+Pulses_bin_3*Difficulty+Pulses_bin_4*Difficulty+Pulses_bin_5*Difficulty+(1|Animal)+(1|Session)

Family: binomial	 Inference: parametric

Number of observations: 1898	 Groups: {'Session': 7.0, 'Animal': 3.0}

Log-likelihood: -1093.506 	 AIC: 2227.013

Random effects:

                Name    Var    Std
Session  (Intercept)  0.003  0.053
Animal   (Intercept)  0.000  0.000

No random effect correlations specified

Fixed effects:

                          Estimate  2.5_ci  97.5_ci     SE     OR  OR_2.5_ci  \
(Intercept)                  1.112   0.782    1.443  0.169  3.041      2.186   
Pulses_bin_1                 0.463   0.125    0.800  0.172  1.588      1.133   
Difficulty1                 -0.944  -1.318   -0.571  0.19

In [552]:
marginal_estimates, comparisons = m4.post_hoc(marginal_vars =["Pulses_bin_5"],grouping_vars = ['Difficulty'], summarize=True)

comparisons

P-values adjusted by tukey method for family of 3 estimates


,Contrast,Estimate,2.5_ci,97.5_ci,SE,DF,Z-stat,P-val,Sig
1,Difficulty2.0 - Difficulty3.0,0.131,-0.325,0.587,0.195,inf,0.671,0.780,
2,Difficulty2.0 - Difficulty6.0,0.556,0.105,1.007,0.193,inf,2.889,0.011,*
3,Difficulty3.0 - Difficulty6.0,0.425,0.064,0.786,0.154,inf,2.762,0.016,*


In [ ]:
#final_df_80 = final_df

from pathlib import Path  
filepath = Path('./LR_with_odor_kernel/4bin_inhalation_kernel_all.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_3b_inhk.to_csv(filepath)

In [ ]:
#dfx = 
df_inhk = pd.concat([df_5b_inhk, df_10b_inhk], ignore_index=True)

filepath = Path('./LR_with_odor_kernel/Weights_inh_kernel_5_10.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_inhk.to_csv(filepath)

In [ ]:
sns.set_style('whitegrid')
sns.set_context('poster')
all_fig_size = (15,5)

final_df_90.set_index('Bin', inplace=True)
final_df_90.groupby('Delay','Dtype','Animal')['Weights'].plot(legend=True)

ValueError: No axis named Dtype for object type DataFrame

In [ ]:
for i in range(all_validated_weights.shape[0]):
    print(animals[i])
    
    plt.plot(np.linspace(0,delay_time/1000,nsegs),all_validated_weights[i].mean(axis=0)/sum(all_validated_weights[i].mean(axis=0))) # /mean_weights.sum()
    plt.plot(np.linspace(0,delay_time/1000,nsegs),all_shuffled_weights[i].mean(axis=0)/sum(all_shuffled_weights[i].mean(axis=0)))
    plt.plot(np.linspace(0,delay_time/1000,nsegs),all_reward_weights[i].mean(axis=0)/sum(all_reward_weights[i].mean(axis=0)))

    plt.xticks(np.linspace(0,delay_time/1000,5))
    plt.ylim(-0.01,0.3)
    plt.show()

NameError: name 'all_validated_weights' is not defined

In [ ]:
sns.set_style('white')
plt.figure(figsize=(6,4))
plt.plot(np.linspace(0,delay_time/1000,nsegs),mean_weights_truth/mean_weights_truth.sum(),'--',color='black')
plt.plot(np.linspace(0,delay_time/1000,nsegs),mean_weights_shuffled/mean_weights_shuffled.sum(),color='grey')
plt.plot(np.linspace(0,delay_time/1000,nsegs),mean_weights/mean_weights.sum(),color='blue')
#plt.plot(np.linspace(0,delay_time/1000,nsegs),np.zeros(nsegs),color='black')
#plt.legend(['truth','shuffled','actual'], loc="best")

plt.legend(['truth','shuffled','actual'])
plt.ylim(0,0.25)
plt.xticks(np.arange(0,10))
plt.xlabel('Time (s)')
#plt.title('3.5s 4:1')
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
#plt.savefig('1to3_2500_ms_logreg_all.svg', bbox_inches='tight')
plt.show()

In [ ]:
#with pd.option_context('display.max_rows', None,
 #                      'display.max_columns', None,
  #                     'display.precision', 3,
                     #  ):

In [ ]:
all_performances.shape[2]

4

In [ ]:
animal_list = ['Banner', 'Natasha', 'Stark', 'Peterparker']
cnt = [2, 3, 4, 6]
delay_times = [1250, 2500, 5000, 10000]

cuky = np.zeros(4)
print(cuky)
all_performances = np.zeros((len(delays),len(cnt),len(animal_list)))
all_performances.shape    
    
for d in range(len(delay_times)):
    trd = get_by_delay(random_trials_all, delay_times[d])
    #print(trd)
    for c in range(len(cnt)):
        trc = get_by_contrast(trd, cnt[c])
        #print(trc)
        for animal, animal_index in zip(animal_list, range(all_performances.shape[2])):
            
            all_performances[d,c,animal_index] = get_performance(get_by_animal(trc, animal))
            
    
print(all_performances)
            

[0. 0. 0. 0.]
[[[0.73233696 0.71692308 0.71543624 0.77809798]
  [0.73229462 0.82843895 0.81073826 0.80537975]
  [0.877429   0.84393939 0.8374613  0.88278932]
  [0.91846154 0.77518248 0.87299035 0.91436865]]

 [[0.79299363 0.80693816 0.77428571 0.77099237]
  [0.89297659 0.80478088 0.81042654 0.80994898]
  [0.87122736 0.93185185 0.81501832 0.88125   ]
  [0.94327391 0.88480801 0.89655172 0.95084746]]

 [[0.77044855 0.70895522 0.74173028 0.76315789]
  [0.80849421 0.85923754 0.81533646 0.8036566 ]
  [0.90964778 0.91413238 0.87436333 0.91298146]
  [0.89502762 0.91076487 0.85542169 0.92200557]]

 [[0.73479319 0.69405478 0.71968709 0.7257384 ]
  [0.77517241 0.7279597  0.74695535 0.86432161]
  [0.86401099 0.8        0.82474227 0.88183161]
  [0.89544688 0.90958409 0.875      0.91839763]]]


In [ ]:
len(get_by_pulses(get_by_contrast(get_by_delay(random_trials_all, 10000), 2), 12,13))

366

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
trials = get_by_contrast(get_by_delay(merged_dataset, 10000), 3)

In [ ]:
# A snip of how Hao was managing the data
x_array_test = np.zeros((nsamples,4))
x_raw_test = np.zeros((nsamples,delay))
for i in range(nsamples):
    x_raw_test[i,:] = get_odor_profile_actual(trials[i])[0:delay]
x_array_test = x_raw_test.reshape((nsamples,4,int(delay/4))).mean(axis=2).squeeze()
print(x_array_test[0])

[0.03109526 0.01554763 0.         0.01554763]


In [ ]:
nsamples=len(trials)

delay=10000
n_bins=5

x_raw = []
trial_n = []
session_n = []
animal_identity = []
trials_lick_side = np.zeros(nsamples)

for i in range(nsamples):
    splitted_x = np.array_split(get_odor_profile_actual(trials[i])[0:delay], n_bins)
    trial_n.append(trials[i].number)
    session_n.append(trials[i].session)
    animal_identity.append(trials[i].name)
    for j in range(len(splitted_x)):
        x_raw.append(splitted_x[j].mean())
        
    trials_lick_side[i] = trials[i].lick_side
    
x_raw = np.asarray(x_raw, dtype="object")
trial_n = np.asarray(trial_n, dtype="object")
session_n = np.asarray(session_n, dtype="object")
animal_identity = np.asarray(animal_identity, dtype="object")
a_id = np.unique(animal_identity, return_inverse=True)[1]

x_raw = x_raw.reshape((nsamples,n_bins))
trial_n = trial_n.reshape((nsamples,-1))
session_n = session_n.reshape((nsamples,-1))
animal_id = a_id.reshape((nsamples,-1))
#print(animal_id.shape, a_id[0:20], a_id[1000:1020])
#print(trials_lick_side)

X1 = np.append(x_raw, animal_id, axis=1)
X2 = np.append(X1, session_n, axis=1)
X3 = np.append(X2, trial_n, axis=1)

#print(X3.shape)
#print(trials_lick_side.shape)

#X3[4309]


In [ ]:
print(x_raw.shape)
print(session_n[0])
print(trials_lick_side.shape)

(5307, 5)
['0']
(5307,)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold 
from sklearn.metrics import accuracy_score

 
acc_score = []
coef = []


logreg = LogisticRegression(random_state=16)
k = 3
kf = KFold(n_splits=k, random_state=None)
nrep=200

for i in tqdm.notebook.trange(nrep):
    ind_array = np.random.permutation(nsamples)
    x_shuffled = X3[ind_array,:]
    z_shuffled = trials_lick_side[ind_array]#.squeeze()


    for train_index , test_index in kf.split(x_shuffled):
        X_train , X_test = x_shuffled[train_index,:],x_shuffled[test_index,:]
        y_train , y_test = z_shuffled[train_index] , z_shuffled[test_index]

        logreg.fit(X_train,y_train)
        pred_values = logreg.predict(X_test)

        acc = accuracy_score(y_test, pred_values)
        acc_score.append(acc)
        coef.append(logreg.coef_.reshape(n_bins+3, -1))

    #avg_acc_score = sum(acc_score)/len(acc_score)


# fit the model with data
#logreg.fit(X_train, y_train)

#coef = logreg.coef_.reshape((n_bins+3, -1))
#y_pred = logreg.predict(X_test)

# from sklearn import metrics

# cnf_matrix = metrics.confusion_matrix(y_test, pred_values)
# cnf_matrix



  0%|          | 0/200 [00:00<?, ?it/s]

C:\Users\Luis\anaconda3\envs\all_behavior\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Luis\anaconda3\envs\all_behavior\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

In [ ]:
print(avg_acc_score)

157.36856981345468


In [ ]:
print(coef)

[array([[ 5.24984653e+00],
       [ 6.82264639e+00],
       [ 7.49432349e+00],
       [ 7.33120178e+00],
       [ 8.97875866e+00],
       [ 1.57535161e-01],
       [-1.47864875e-01],
       [-2.40633580e-03]]), array([[ 5.91282255e+00],
       [ 6.79415078e+00],
       [ 7.58156831e+00],
       [ 8.10731791e+00],
       [ 8.53016919e+00],
       [ 1.73522314e-01],
       [-1.48313685e-01],
       [-2.41743314e-03]]), array([[ 6.19501273e+00],
       [ 6.25577609e+00],
       [ 7.34470266e+00],
       [ 8.01658859e+00],
       [ 8.37587621e+00],
       [ 1.64504718e-01],
       [-1.44034055e-01],
       [-1.81556547e-03]]), array([[ 5.40143689e+00],
       [ 6.70582680e+00],
       [ 7.33121177e+00],
       [ 7.46980673e+00],
       [ 8.70018297e+00],
       [ 1.65719137e-01],
       [-1.43847836e-01],
       [-2.23026231e-03]]), array([[ 6.03945413e+00],
       [ 6.57193728e+00],
       [ 7.64983996e+00],
       [ 7.69631146e+00],
       [ 8.35613846e+00],
       [ 1.61933637e-01],
   

In [ ]:
ccc = np.asarray(coef, dtype="object").reshape(600,n_bins+3)


In [ ]:
ddd = ccc.mean(axis=0)
normw = sum(ddd[0:5])
print(w)

[0.16216431546472396 0.18407304732095567 0.20525407703947893
 0.21412312632210775 0.23438543385273372]


In [ ]:
print(coef[0])

[[ 8.26962494e+00]
 [ 9.89973873e+00]
 [ 1.12844967e+01]
 [ 1.58555648e-01]
 [-1.20362603e-01]
 [-1.88088910e-03]]


In [ ]:
trialx = get_by_animal(get_by_contrast(get_by_delay(random_trials_all, 10000), 3), 'Siamese')
print(len(trialx))

506


In [ ]:
def validator(trials,n_reps,n_bins): # x_array (odor profile passed through the kernel) 
        #prediction on z_array (licking decision)

    models = list()
    avg_acc = 0
    avg_acc_test = 0
    avg_cost = 0
    for i in tqdm.notebook.trange(n_reps):
        x_raw = []
        trial_n = []
        session_n = []
        animal_identity = []
        trials_lick_side = np.zeros(nsamples)
        ind_array = np.random.permutation(nsamples)
        x_splitted = np.array_split(get_odor_profile_actual(trials[i])[0:delay], n_bins)
        for j in range(len(x_splitted)):
            x_raw.append(x_splitted[j].mean())
        trial_n.append(trials[ind_array].number)
        session_n.append(trials[ind_array].session)
        animal_identity.append(trials[ind_array].name)
        x_raw = np.asarray(x_raw, dtype="object")
        trial_n = np.asarray(trial_n, dtype="object")
        session_n = np.asarray(session_n, dtype="object")
        animal_identity = np.asarray(animal_identity, dtype="object")
        a_id = np.unique(animal_identity, return_inverse=True)[1]
        

        x_reshaped = x_raw.reshape((nsamples,n_bins))
        x_shuffled[i] = x_shuffled[ind_array]
        trial_n = trial_n.reshape((nsamples,-1))
        session_n = session_n.reshape((nsamples,-1))
        animal_id = a_id.reshape((nsamples,-1))
        
        X1 = np.append(x_shuffled, animal_id, axis=1)
        X2 = np.append(X1, session_n, axis=1)
        X = np.append(X2, trial_n, axis=1)
        
        return X
        
        
        
#         z_shuffled = trials[ind_array].lick_side.squeeze()
#         x_all = Tensor(x_shuffled.astype(float)).to(self.device)
#         z_all = Tensor(z_shuffled.astype(float)).to(self.device)
#         z_all = z_all.long().to(self.device)
#         x_train = x_all[0:int(self.nsamples*self.trainsize),:]
#         z_train = z_all[0:int(self.nsamples*self.trainsize)]
#         x_test = x_all[int(self.nsamples*self.trainsize):,:]
#         z_test = z_all[int(self.nsamples*self.trainsize):]
#         model, acc, acc_test,cost = self.main(target,x_test,z_test,x_train,z_train)
#         model.to(device)
#         avg_acc += acc
#         avg_acc_test += acc_test
#         avg_cost += cost
#         models.append(model)

#     avg_acc/=n
#     avg_acc_test/=n
#     avg_cost/=n

#     print("")
#     print('validation rounds =',i+1,'train performance = %.2f%%'%avg_acc,
#           'test performance = %.2f%%' %avg_acc_test,'cost = %.2f' %avg_cost)
#     return models

In [ ]:
validator(trialx,1,3)

  0%|          | 0/1 [00:00<?, ?it/s]

TypeError: only integer scalar arrays can be converted to a scalar index